In [4]:
import requests

import cv2
import numpy
from matplotlib import pyplot

def show_image(img):
  width = 15;
  height = img.shape[0] // (img.shape[1] / width);
  pyplot.figure(figsize=(width, height))
  pyplot.axis("off")
  pyplot.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB));

In [1]:
runspaces_root = 'F:\Projects\skillbox.python.webinar.2020-12-14.object.detection\Runspace'
print('runspaces_root:', runspaces_root)

import uuid
import os
runspace_root = os.path.join(runspaces_root, str(uuid.uuid4()))
print('runspace_root:', runspace_root)
os.makedirs(runspace_root)

runspaces_root: F:\Projects\skillbox.python.webinar.2020-12-14.object.detection\Runspace
runspace_root: F:\Projects\skillbox.python.webinar.2020-12-14.object.detection\Runspace\f88d9c51-2dc8-421d-978f-97db8dcc088c


In [8]:
yolo_weights_url = 'https://pjreddie.com/media/files/yolov3.weights'
#yolo_weights_url = 'https://pjreddie.com/media/files/yolov3-tiny.weights'
yolo_weights_path = os.path.join(runspace_root, 'yolo.weights')
print('yolo_weights_path:', yolo_weights_path)
open(yolo_weights_path, 'wb').write(requests.get(yolo_weights_url, allow_redirects = True).content)

yolo_weights_path: F:\Projects\skillbox.python.webinar.2020-12-14.object.detection\Runspace\f88d9c51-2dc8-421d-978f-97db8dcc088c\yolo.weights


248007048

In [9]:
yolo_config_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg'
#yolo_config_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3-tiny.cfg'
yolo_config_path = os.path.join(runspace_root, 'yolo.cfg')
print("yolo_config_path:", yolo_config_path)
open(yolo_config_path, 'wb').write(requests.get(yolo_config_url, allow_redirects = True).content)
yolo_frame = (608, 608)
#yolo_frame = (416, 416)

yolo_config_path: F:\Projects\skillbox.python.webinar.2020-12-14.object.detection\Runspace\f88d9c51-2dc8-421d-978f-97db8dcc088c\yolo.cfg


In [10]:
coco_names_url = 'https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names'
coco_names_path = os.path.join(runspace_root, 'coco.names')
print('coco_names_path:', coco_names_path)
open(coco_names_path, 'wb').write(requests.get(coco_names_url, allow_redirects = True).content)

objects = [line for line in open(coco_names_path, 'r').read().split('\n') if line]
colors = numpy.random.uniform(0, 255, size=(len(objects), 3))

coco_names_path: F:\Projects\skillbox.python.webinar.2020-12-14.object.detection\Runspace\f88d9c51-2dc8-421d-978f-97db8dcc088c\coco.names


In [65]:
ai = cv2.dnn.readNet(yolo_config_path, yolo_weights_path)
layers_names = ai.getLayerNames()
output_layers = [layers_names[i[0] - 1] for i in ai.getUnconnectedOutLayers()]

def draw_object(img, to_draw):
  width = 1;
  color = to_draw['class']['color']
  
  pt1 = (to_draw['box'][0], to_draw['box'][1])
  pt2 = (pt1[0] + to_draw['box'][2], pt1[1] + to_draw['box'][2])
  img = cv2.rectangle(img, pt1, pt2, color, width)
  #
  #pt3 = (pt1[0], pt1[1])
  #pt4 = (pt2[0], pt1[1] - 40)
  #img = cv2.rectangle(img, pt3, pt4, color, width)
  #img = cv2.rectangle(img, pt3, pt4, color, -1)
  #
  #pt5 = (pt3[0] + 10, pt4[1] + 30)
  #img = cv2.putText(img, "{0} ({1:.0%})".format(to_draw['class']['name'], to_draw['confidence']), pt5, cv2.FONT_HERSHEY_SIMPLEX, 1, [255 - x for x in color], 3)
  
  return img;

def get_detected_objects_in_frame(frame, confidence_threshold = 0.8, filter_by_class = lambda x: True):
  centers = [];
  boxes = [];
  confs = [];
  class_ids = [];

  height = frame.shape[0]
  width = frame.shape[1]

  ai.setInput(cv2.dnn.blobFromImage(frame, 1/255, yolo_frame, (0, 0, 0), swapRB = True, crop = False))
  outputs = ai.forward(output_layers)
  for output in outputs:
    for detect in output:
      scores = detect[5:]
      class_id = numpy.argmax(scores)
      conf = scores[class_id] * 1.0
      if conf >= confidence_threshold:
        if filter_by_class(objects[class_id]):
          center_x = int(detect[0] * width)
          center_y = int(detect[1] * height)
          
          centers.append([center_x, center_y]);
          
          w = int(detect[2] * width)
          h = int(detect[3] * height)
          x = int(center_x - w / 2)
          y = int(center_y - h / 2)
        
          boxes.append([x, y, w, h])
          confs.append(float(conf))
          class_ids.append(class_id)
  # filter out NMS probes
  centers, boxes, confs, class_ids = [[arr[i] for i in [x[0] for x in cv2.dnn.NMSBoxes(boxes, confs, 0, 0.45)]] for arr in [centers, boxes, confs, class_ids]];

  detected_objects = [{'center': centers[idx], 'box': boxes[idx], 'class': {'name': objects[class_ids[idx]], 'color': colors[class_ids[idx]]}, 'confidence': confs[idx]} for idx in range(0, len(boxes))]
  return detected_objects

def get_detected_objects_impl(img, confidence_threshold = 0.8, filter_by_class = lambda x: True):
  # first, pass over whole image
  detected_objects = get_detected_objects_in_frame(img, confidence_threshold, filter_by_class);
  # second, split image in frames and pass one by one
  if(img.shape[0] > yolo_frame[1] or img.shape[1] > yolo_frame[0]):
    frame_height = yolo_frame[1];
    frame_height_half = frame_height // 2;
    frames_per_height = img.shape[0] // frame_height_half;
    frame_width = yolo_frame[0];
    frame_width_half = frame_width // 2;
    frames_per_width = img.shape[1] // frame_width_half;
    for i in range(frames_per_height):
      frame_x1 = i * frame_height_half;
      frame_x2 = min(frame_x1 + frame_height, img.shape[0])
      for j in range(frames_per_width):
        frame_y1 = j * frame_width_half;
        frame_y2 = min(frame_y1 + frame_width, img.shape[1])
        print(i, frame_x1, frame_x2, j, frame_y1, frame_y2)
        frame = img[frame_x1:frame_x2, frame_y1:frame_y2]
        # detect objects in frame
        detected_objects_from_frame = get_detected_objects_in_frame(frame, confidence_threshold, filter_by_class);
        # adjust according to frame start
        for obj in detected_objects_from_frame:
          obj['center'][0] += frame_y1;
          obj['center'][1] += frame_x1;
          obj['box'][0] += frame_y1;
          obj['box'][1] += frame_x1
        # mix up
        detected_objects += detected_objects_from_frame;
    ## nms with already detected
    #detected_objects = [detected_objects[i] for i in [x[0] for x in cv2.dnn.NMSBoxes([obj['box'] for obj in detected_objects], [obj['confidence'] for obj in detected_objects], 0, 0.1)]]

  return detected_objects;

def get_detected_objects(img, downscale = 2, confidence_threshold = 0.8, filter_by_class = lambda x: True):
  img_to_detect = img;
  if(downscale != 1):
    img_to_detect = cv2.resize(img, (int(img.shape[1] / downscale), int(img.shape[0] / downscale)));

  detected_objects = get_detected_objects_impl(img_to_detect, confidence_threshold, filter_by_class)
  # adjust for downscale
  for obj in detected_objects:
    if(downscale != 1):
      obj['center'] = [int(x * downscale) for x in obj['center']];
      obj['box'] = [int(x * downscale) for x in obj['box']];
  return detected_objects;

def apply_yolo(img, detection_downscale = 1.1, detection_confidence_threshold = 0.8, detection_filter_by_class = lambda x: True):
  objects_detected = get_detected_objects(img, detection_downscale, detection_confidence_threshold, detection_filter_by_class);
  for obj in objects_detected:
    # draw box
    img = draw_object(img, obj)
  return img;


In [14]:
img_path = os.path.join(runspace_root, 'keszthelyi-timi-P25wpQ8ndl8-unsplash.jpg')
print('img_path:', img_path)
img_edited_path = os.path.join(runspace_root, 'keszthelyi-timi-P25wpQ8ndl8-unsplash_edited.jpg')
print('img_edited_path:', img_edited_path)

#img_url = 'https://images.unsplash.com/photo-1589828155685-83225f7d91f3?ixid=MXwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHw%3D&ixlib=rb-1.2.1&auto=format&fit=crop&w=2153&q=80'
img_url = 'https://images.unsplash.com/photo-1595365409261-4f1c3a8c9282?ixid=MXwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHw%3D&ixlib=rb-1.2.1&auto=format&fit=crop&w=1950&q=80'
open(img_path, 'wb').write(requests.get(img_url, allow_redirects = True).content)

img_path: F:\Projects\skillbox.python.webinar.2020-12-14.object.detection\Runspace\f88d9c51-2dc8-421d-978f-97db8dcc088c\keszthelyi-timi-P25wpQ8ndl8-unsplash.jpg
img_edited_path: F:\Projects\skillbox.python.webinar.2020-12-14.object.detection\Runspace\f88d9c51-2dc8-421d-978f-97db8dcc088c\keszthelyi-timi-P25wpQ8ndl8-unsplash_edited.jpg


754023

In [ ]:
img = cv2.imread(img_path)
img = apply_yolo(img, 5, 0.2, lambda obj_class: obj_class == 'car')
show_image(img)